<a href="https://colab.research.google.com/github/LeeDongN/Medica_AI2/blob/main/final_data_label.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code read a folder of data. After applying preprocessing (DC component removal), it will extract number of maximas specified by a number and the labels will be assigned through final_function. The data is then balanced by ADASYN and SMOTE oversampling methods. At the end, KNN is used as classification approach. The accuracies will be compared with confusion matrix at the end.

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
from matplotlib import pyplot as plt
import os
import sklearn
from google.colab import drive
from scipy.signal import find_peaks
drive.mount('/content/drive')
path = '/content/drive/MyDrive/new_data/1stSeries_07.05.21'

Mounted at /content/drive


# functions

#### importing first data function

In [ ]:
def first_data_import():
  # import the first_data
  # first_data making
  first_data = []
  first_data = pd.read_csv('/content/drive/My Drive/data2/REF.CSV', skiprows=24)
  first_data_origin = pd.read_csv('/content/drive/My Drive/data2/REF.CSV', skiprows=24)
  first_a = float(first_data_origin.columns[0])
  first_b = float(first_data_origin.columns[1])
  first_data.loc[0] = [first_a, first_b]
  first_data.columns=['Wave_length', 'Amplitude']
  first_data['label']=1.3
  first_np = first_data[['Wave_length', 'Amplitude']].to_numpy()
  first_np = np.ravel(first_np, order='F')
  first_np = np.array([first_np])

  return first_data, first_np

#### importing whole data and stack function

In [ ]:
import os
import numpy as np
import pandas as pd

def stack_data(path):
    stack_np = None  # Initialize as None to properly concatenate later
    folder_list = os.listdir(path)

    for folder_name in folder_list:
        temp_folder_list = os.listdir(os.path.join(path, folder_name))

        for file_name in temp_folder_list:
            # Read the data into a Pandas DataFrame
            file_path = os.path.join(path, folder_name, file_name)
            data = pd.read_csv(file_path, skiprows=24)

            # Extract the first two columns and rename them
            data = data.iloc[:, :2]
            data.columns = ['Wave_length', 'Amplitude']

            # Stack the data as a single row
            temp = data.values.ravel(order='F')  # Convert DataFrame to NumPy array and flatten
            temp = temp.reshape(1, -1)  # Reshape to a 2D array with a single row

            # Concatenate to stack_np
            if stack_np is None:
                stack_np = temp
            else:
                stack_np = np.concatenate((stack_np, temp), axis=1)

    return stack_np


In [ ]:
stack_np=stack_data(path)
stack_np.shape

(1, 3162600)

#### removing DC function

In [ ]:
# Implementing a simple DC removal function
def remove_dc(signal):
    # Calculate the mean of the signal
    signal_mean = np.mean(signal)
    # Remove the mean from the signal
    dc_removed_signal = signal - signal_mean
    return dc_removed_signal

In [ ]:
dc_removed_signal = remove_dc(stack_np)

#### finding peaks function

In [ ]:
# Find peaks
def detect_peaks(signal, threshold=None, min_distance=1, prominence=None):
    if threshold is None:
        # Set threshold as a percentage of the maximum peak
        max_peak = np.max(signal)
    peaks, _ = find_peaks(signal, height=threshold, distance=min_distance, prominence=prominence)
    return peaks

#### peak values extracting function

In [ ]:
def extract_peaks(X, Y, peak_range):
  # pandas to numpy
  peak_X = X.to_numpy()
  peak_Y = Y.to_numpy()
  # extract the peak value index
  index = list(peak_Y).index(max(peak_Y))

  # temp saving
  extracted_x = []
  extracted_y = []

  # extract the values from highest peak, and surrounding peaks
  for k in range(peak_range):
    # index labeling
    i = k + 1
    p_temp_index = index + i
    m_temp_index = index - i
    # extract y
    extracted_y.append(peak_Y[p_temp_index])
    extracted_y.append(peak_Y[m_temp_index])
    # extract X
    extracted_x.append(peak_X[p_temp_index])
    extracted_x.append(peak_X[m_temp_index])

  # extrack the center(highest) value
  extracted_x.append(peak_X[index])
  extracted_y.append(peak_Y[index])

  return extracted_x, extracted_y

In [ ]:
extracted_np_data, label = final_function(path,5)
extracted_np_data.shape

(251, 22)

#Final Function for data with two labels

In [ ]:
def final_function(path, peak_range):
    extracted_data = []  # Container for extracted peak data
    label = []  # Container for labels

    folder_list = os.listdir(path)

    for folder_name in folder_list:
        temp_folder_list = os.listdir(os.path.join(path, folder_name))

        for file_name in temp_folder_list:
            # Assign label based on folder name
            label.append(1 if float(folder_name) > 1.36 else 0)

            # Read the data into a Pandas DataFrame
            file_path = os.path.join(path, folder_name, file_name)
            data = pd.read_csv(file_path, skiprows=24)
            data_origin = pd.read_csv(file_path, skiprows=24)

            # Restore the removed data (the first data)
            a = float(data_origin.columns[0])
            b = float(data_origin.columns[1])
            data.loc[0] = [a, b]
            data.columns = ['Wave_length', 'Amplitude']

            # Remove DC offset
            dc_removed_Y = remove_dc(data['Amplitude'])

            # Detect peaks
            min_distance = 100  # Set the minimum distance between peaks
            local_maxima_indices = detect_peaks(dc_removed_Y, min_distance=min_distance)
            local_maxima_x = data['Wave_length'][local_maxima_indices]
            local_maxima_y = dc_removed_Y[local_maxima_indices]

            # Extract peaks within the specified range
            extracted_x, extracted_y = extract_peaks(local_maxima_x, local_maxima_y, peak_range)

            # Concatenate and append to extracted_data
            temp = np.concatenate((extracted_x, extracted_y))
            extracted_data.append(temp)

    # Convert lists to numpy arrays
    extracted_np_data = np.array(extracted_data)
    label = np.array(label)

    return extracted_np_data, label


#Final Function for Data with 5 labels

In [ ]:
def final_function(path, peak_range):
    extracted_data = []  # Container for extracted peak data
    label = []  # Container for labels

    folder_list = os.listdir(path)

    for folder_name in folder_list:
        temp_folder_list = os.listdir(os.path.join(path, folder_name))

        for file_name in temp_folder_list:
            # Assign label based on folder name range
            folder_value = float(folder_name)
            if folder_value == 1.3:
                label.append(0)
            elif 1.3 <= folder_value < 1.32:
                label.append(1)
            elif 1.32 <= folder_value < 1.38:
                label.append(2)
            elif 1.38 <= folder_value < 1.39:
                label.append(3)
            elif folder_value >= 1.39:
                label.append(4)

            # Read the data into a Pandas DataFrame
            file_path = os.path.join(path, folder_name, file_name)
            data = pd.read_csv(file_path, skiprows=24)
            data_origin = pd.read_csv(file_path, skiprows=24)

            # Restore the removed data (the first data)
            a = float(data_origin.columns[0])
            b = float(data_origin.columns[1])
            data.loc[0] = [a, b]
            data.columns = ['Wave_length', 'Amplitude']

            # Remove DC offset
            dc_removed_Y = remove_dc(data['Amplitude'])

            # Detect peaks
            min_distance = 100  # Set the minimum distance between peaks
            local_maxima_indices = detect_peaks(dc_removed_Y, min_distance=min_distance)
            local_maxima_x = data['Wave_length'][local_maxima_indices]
            local_maxima_y = dc_removed_Y[local_maxima_indices]

            # Extract peaks within the specified range
            extracted_x, extracted_y = extract_peaks(local_maxima_x, local_maxima_y, peak_range)

            # Concatenate and append to extracted_data
            temp = np.concatenate((extracted_x, extracted_y))
            extracted_data.append(temp)

    # Convert lists to numpy arrays
    extracted_np_data = np.array(extracted_data)
    label_np = np.array(label)

    return extracted_np_data, label_np


In [ ]:
extracted_np_data,label_np = final_function(path,8)
np.save("/content/drive/MyDrive/newdata_1stSeries.npy", extracted_np_data)
np.save("/content/drive/MyDrive/5labels.npy", label)

# Test

In [ ]:
extracted_np_data,label_np = final_function(path,10)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

data = extracted_np_data
labels = label_np

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Initialize KNN classifier
knn = KNeighborsClassifier(n_neighbors=8)

# Train the KNN classifier
knn.fit(X_train, y_train)

# Evaluate the KNN classifier
accuracy = knn.score(X_test, y_test)

# Predict labels for the test set
y_pred = knn.predict(X_test)

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"-----------------------------------Accuracy before Oversampling-----------------------------------")
print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(conf_matrix)

# Calculate accuracy for each class
class_accuracy = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

for i, acc in enumerate(class_accuracy):
    print(f"Accuracy for class {i}: {acc:.2f}")


-----------------------------------Accuracy before Oversampling-----------------------------------
Accuracy: 0.8627450980392157
Confusion Matrix:
[[ 2  0  0  0  0]
 [ 1  1  0  0  0]
 [ 0  1  9  1  3]
 [ 0  0  0  2  0]
 [ 0  0  1  0 30]]
Accuracy for class 0: 1.00
Accuracy for class 1: 0.50
Accuracy for class 2: 0.64
Accuracy for class 3: 1.00
Accuracy for class 4: 0.97


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE, ADASYN
from collections import Counter

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Display class distribution before oversampling
print("Class distribution before oversampling:", sorted(Counter(y_train).items()))

# Apply SMOTE oversampling
X_resampled_smote, y_resampled_smote = SMOTE().fit_resample(X_train, y_train)

# Display class distribution after SMOTE oversampling
print("Class distribution after SMOTE:", sorted(Counter(y_resampled_smote).items()))

# Apply ADASYN oversampling
X_resampled_adasyn, y_resampled_adasyn = ADASYN().fit_resample(X_train, y_train)

# Display class distribution after ADASYN oversampling
print("Class distribution after ADASYN:", sorted(Counter(y_resampled_adasyn).items()))

# Initialize KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)

# Train the KNN classifier with SMOTE oversampled data
knn.fit(X_resampled_smote, y_resampled_smote)

# Evaluate the KNN classifier with SMOTE oversampled data
y_pred_smote = knn.predict(X_test)
accuracy_smote = accuracy_score(y_test, y_pred_smote)
print("Overall Accuracy with SMOTE:", accuracy_smote)

# Calculate accuracy for each class with SMOTE oversampled data
class_accuracy_smote = conf_matrix_smote.diagonal() / conf_matrix_smote.sum(axis=1)
print("Accuracy for each class with SMOTE oversampling:")
for i, acc in enumerate(class_accuracy_smote):
    print(f"Class {i}: {acc:.2f}")

# Calculate the confusion matrix with SMOTE oversampled data
conf_matrix_smote = confusion_matrix(y_test, y_pred_smote)
print("Confusion Matrix with SMOTE:")
print(conf_matrix_smote)

# Train the KNN classifier with ADASYN oversampled data
knn.fit(X_resampled_adasyn, y_resampled_adasyn)

## Evaluate the KNN classifier with ADASYN oversampled data
#y_pred_adasyn = knn.predict(X_test)
#accuracy_adasyn = accuracy_score(y_test, y_pred_adasyn)
#print("Overall Accuracy with ADASYN:", accuracy_adasyn)
## Calculate accuracy for each class with ADASYN oversampled data
#class_accuracy_adasyn = conf_matrix_adasyn.diagonal() / conf_matrix_adasyn.sum(axis=1)
#print("Accuracy for each class with ADASYN oversampling:")
#for i, acc in enumerate(class_accuracy_adasyn):
#    print(f"Class {i}: {acc:.2f}")

## Calculate the confusion matrix with ADASYN oversampled data
#conf_matrix_adasyn = confusion_matrix(y_test, y_pred_adasyn)
#print("Confusion Matrix with ADASYN:")
#print(conf_matrix_adasyn)



Class distribution before oversampling: [(0, 9), (1, 10), (2, 58), (3, 10), (4, 113)]
Class distribution after SMOTE: [(0, 113), (1, 113), (2, 113), (3, 113), (4, 113)]
Class distribution after ADASYN: [(0, 114), (1, 112), (2, 113), (3, 111), (4, 113)]
Overall Accuracy with SMOTE: 0.7450980392156863
Accuracy for each class with SMOTE oversampling:
Class 0: 1.00
Class 1: 0.50
Class 2: 0.64
Class 3: 1.00
Class 4: 0.87
Confusion Matrix with SMOTE:
[[ 2  0  0  0  0]
 [ 1  1  0  0  0]
 [ 2  2  7  3  0]
 [ 0  0  0  2  0]
 [ 0  0  2  3 26]]


KNeighborsClassifier()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE, ADASYN
from collections import Counter

# Apply SMOTE oversampling to the entire dataset
X_resampled_smote, y_resampled_smote = SMOTE().fit_resample(data, labels)

# Display class distribution after SMOTE oversampling
print("Class distribution after SMOTE:", sorted(Counter(y_resampled_smote).items()))

# Apply ADASYN oversampling to the entire dataset
X_resampled_adasyn, y_resampled_adasyn = ADASYN().fit_resample(data, labels)

# Display class distribution after ADASYN oversampling
print("Class distribution after ADASYN:", sorted(Counter(y_resampled_adasyn).items()))

# Split the SMOTE oversampled data into training and testing sets
X_train_smote, X_test_smote, y_train_smote, y_test_smote = train_test_split(X_resampled_smote, y_resampled_smote, test_size=0.2, random_state=42)

# Split the ADASYN oversampled data into training and testing sets
X_train_adasyn, X_test_adasyn, y_train_adasyn, y_test_adasyn = train_test_split(X_resampled_adasyn, y_resampled_adasyn, test_size=0.2, random_state=42)

# Initialize KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)

# Train the KNN classifier with SMOTE oversampled data
knn.fit(X_train_smote, y_train_smote)

# Evaluate the KNN classifier with SMOTE oversampled test data
y_pred_smote = knn.predict(X_test_smote)
accuracy_smote = accuracy_score(y_test_smote, y_pred_smote)
print("Overall Accuracy with SMOTE:", accuracy_smote)

# Calculate the confusion matrix with SMOTE oversampled test data
conf_matrix_smote = confusion_matrix(y_test_smote, y_pred_smote)
print("Confusion Matrix with SMOTE:")
print(conf_matrix_smote)

# Train the KNN classifier with ADASYN oversampled data
knn.fit(X_train_adasyn, y_train_adasyn)

# Evaluate the KNN classifier with ADASYN oversampled test data
y_pred_adasyn = knn.predict(X_test_adasyn)
accuracy_adasyn = accuracy_score(y_test_adasyn, y_pred_adasyn)
print("Overall Accuracy with ADASYN:", accuracy_adasyn)

# Calculate the confusion matrix with ADASYN oversampled test data
conf_matrix_adasyn = confusion_matrix(y_test_adasyn, y_pred_adasyn)
print("Confusion Matrix with ADASYN:")
print(conf_matrix_adasyn)


Class distribution after SMOTE: [(0, 144), (1, 144), (2, 144), (3, 144), (4, 144)]
Class distribution after ADASYN: [(0, 147), (1, 145), (2, 142), (3, 145), (4, 144)]
Overall Accuracy with SMOTE: 0.9236111111111112
Confusion Matrix with SMOTE:
[[30  0  1  0  0]
 [ 2 22  0  0  0]
 [ 0  1 23  0  1]
 [ 1  0  0 23  3]
 [ 0  1  0  1 35]]
Overall Accuracy with ADASYN: 0.9310344827586207
Confusion Matrix with ADASYN:
[[24  4  0  0  0]
 [ 0 26  1  0  0]
 [ 0  1 23  1  0]
 [ 0  0  0 29  0]
 [ 0  0  1  2 33]]
